# Module 7. Clustering
# Introduction to clustering
## Lecture objectives
1. Examine the purposes and potential uses of clustering
2. Introduce more exploratory data analysis techniques, such as pairplots
3. Demonstrate how to implement k-means clustering

## Why cluster?
Cluster analysis is an exploratory technique to identify sensible groupings in a dataset. The analyst has no prior knowledge of what these clusters are, and the data are not labeled with the "correct" cluster. Thus, cluster analysis is an *unsupervised* machine learning technique.

Some potential applications of clustering:
* Identify types of Marine Protected Area (e.g., [Bohorquez et al. 2019](https://www.sciencedirect.com/science/article/pii/S0308597X19304439))
* Identify types of street networks (e.g., [Barrington-Leigh and Millard-Ball 2020](https://www.pnas.org/content/117/4/1941))
* Identify types of neighborhood (e.g., [Kendig 2007](https://www.tandfonline.com/doi/abs/10.1080/01944367608977731))
* Identify types of transit agencies (e.g. [Ederer et al. 2019](https://journals.sagepub.com/doi/full/10.1177/0361198119852074))
* Identify patterns of cruising for parking (e.g. [Millard-Ball, Weinberger & Hampshire 2021](https://findingspress.org/article/28061-the-shape-of-cruising))

Clustering the data in this way can help you see regularities in the data that you can then interpret. It might suggest policies that are appropriate for one group of cities or agencies but not another. Or it could identify a peer group against which to benchmark (say) affordable housing construction costs or transit on-time performance.

## Types of clustering
Formally, clustering takes a set of *N* objects and finds *K* groups based on a measure of similarity. For a technical yet accessible overview, I recommend [Jain 2010](https://www.sciencedirect.com/science/article/abs/pii/S0167865509002323). 

The two broad groups of clustering algorithms are *hierarchical* and *partitional*. 

Ederer et al. (2019), for example, use a hierarchical algorithm to classify transit agencies.

<img src="https://journals.sagepub.com/na101/home/literatum/publisher/sage/journals/content/trra/2019/trra_2673_11/0361198119852074/20191218/images/medium/10.1177_0361198119852074-fig1.gif" style="width:50%;"/>


But let's start with a partitional algorithm. The most popular is called *k-means*. Again, this is an exploratory data analysis process. The analyst needs to specify the number of clusters *K*, and should experiment with different values of *K* until a meaningful grouping emerges. Another way to choose *K* is the [elbow method](https://en.wikipedia.org/wiki/Elbow_method_(clustering)), but we won't discuss that here.

## Example: precinct-level voting
We'll use the `sklearn` library to implement the k-means algorithm. The aim: identify a typology of voters based on precinct-level data.

The California [Statewide Database](https://statewidedatabase.org), maintained by UC Berkeley, provides access to voting data. Your GitHub repository should include the November 2020 precinct data for Los Angeles County.

In [ ]:
import pandas as pd

df = pd.read_csv('../data/c037_g20_sov_data_by_g20_srprec.csv')
df.head()

The unique identifier is given by the precinct column, `srprec`, so let's set that as the index.

In [ ]:
df.set_index('srprec', inplace=True)
df.index.is_unique  # verify that it's unique

What columns are in the data set?

In [ ]:
df.columns

There are obviously a lot of columns. You can see the [full codebook here](https://statewidedatabase.org/d10/g20.html). But the propositions are pretty self explanatory. `PRSDEM01` gives votes for Biden, `PRSREP01` for Trump, etc. Note the state Senate and Assembly races will have different candidates depending on the precinct, so let's ignore those.

What do the data in these columns look like?

In [ ]:
df[['PRSDEM01','PRSREP01']].head()

We see that the numbers are in absolute terms. Let's convert them to vote share.

<div class="alert alert-block alert-info">
<strong>Exercise:</strong> How would you create a new column with percentage vote share for Biden and Trump?
</div>

In [ ]:
# this is share of two-party vote (ignoring other candidates)
df['Biden_pc'] = df.PRSDEM01 / (df.PRSDEM01+df.PRSREP01)*100

Let's do the same for each proposition too. 

How can we get a list with the numbers of the propositions? We'll use a list comprehension to get the list of relevant columns.

In [ ]:
props = [col for col in df.columns if col.startswith('PR_') and col.endswith('Y')]
print(props)

And another list comprehension to get the relevant proposition numbers. Note that these are always the 4th and 5th characters.

In [ ]:
# we can use our string indexing to get the 4th and 5th characters
# for example
print('PR_14_Y'[3:5])

# apply this in a list comprehension
props = [prop[3:5] for prop in props]
print(props)

# we could have done this in one go
print([col[3:5] for col in df.columns if col.startswith('PR_') and col.endswith('Y')])

Now, let's loop over this list of propositions to calculate the vote share.

In [ ]:
for prop in props:
    df[prop+'_pc_yes'] = df['PR_'+prop+'_Y'] / (df['PR_'+prop+'_Y'] 
                                              + df['PR_'+prop+'_N'])*100
df.head()

### Inspect and standardize the data

Number 1 rule: before applying any algorithm to your data, look at it!

We could create a scatterplot matrix manually. But `seaborn` has a [nice function to do this for us](https://seaborn.pydata.org/examples/scatterplot_matrix.html).

In [ ]:
import seaborn as sns

sns.pairplot?

We'll plot a subset of the columns, ignoring a few of the less critical propositions. Should stem cell research and dialysis rules really be on the ballot?

In [ ]:
# get a list of all the columns with 'pc' in the name
cols_to_plot = [col for col in df.columns if '_pc' in col]

# remove those we don't want
cols_to_plot.remove('14_pc_yes') 
cols_to_plot.remove('23_pc_yes') 
cols_to_plot.remove('24_pc_yes') 

# kind='reg' adds the line of best fit
ax = sns.pairplot(df[cols_to_plot], kind='reg')  

There are pretty strong relationships between Presidential voting and the propositions. All have a positive correlation except for Prop 20 (harsher sentencing) and Prop 22 (independent contractor status for drivers for Uber, Doordash, etc.). [A helpful reminder of the propositions is here](https://ballotpedia.org/California_2020_ballot_propositions).

But there isn't a perfect relationship. Perhaps cluster analysis can reveal some groupings? In other words, do precincts cluster according to ideological "types"?

First, it helps to pre-process the data in two ways:
* Let's align the data so that a higher percent means more progressive. This means using the percent "no" for Props 20 and 22
* We should standardize each variable to mean zero and standard deviation one. This helps ensure that the distances in multidimensional space are consistent. (Since we have a percentage measure, it won't make much difference compared to a variable like population, but it's good practice.)

In [ ]:
for prop in ['20','22']:
    df[prop+'_pc_no'] = 100 - df[prop+'_pc_yes']
    df.drop(columns=[prop+'_pc_yes'], inplace=True)

# then rerun the same code as above
cols_to_plot = [col for col in df.columns if '_pc' in col]
cols_to_plot.remove('14_pc_yes') 
cols_to_plot.remove('23_pc_yes') 
cols_to_plot.remove('24_pc_yes') 

# see https://scikit-learn.org/stable/modules/preprocessing.html for standardization
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(df[cols_to_plot])

# as in the previous lecture, 
# the scaler returns a numpy array, so we cast this as a DataFrame 
# and need to specify the column names and index
df_scaled = pd.DataFrame(scaler.transform(df[cols_to_plot]), 
                         columns=cols_to_plot, index=df.index)

<div class="alert alert-block alert-info">
<strong>Exercise:</strong> Unpack each step in the previous lines of code, and find out what is returned by <strong>scaler</strong> and <strong>scaler.transform(df[cols_to_plot])</strong>.
</div>

Let's check that our data still look reasonable by rerunning the same pairplot.

Notice that the y axes run from about -3 to +3. This should be true for any standardized variable, as most observations are within 3 standard deviations of the mean.

In [ ]:
ax = sns.pairplot(df_scaled, kind='reg')

### KMeans in scikit-learn
As always, the data wrangling was a large part of our work. Now, we are ready to do the cluster analysis, which is much simpler.

The documention has some useful examples.

In [ ]:
from sklearn.cluster import KMeans
KMeans?

We can specify the number of clusters. Optionally, we can specify the random state, so that we can reproduce our work. 

Then we fit to our data, in this case `df_scaled`.

In [ ]:
kmeans = KMeans(n_clusters=5, random_state=0).fit(df_scaled)

Here, the error message is pretty helpful. Let's drop the NaNs.

In [ ]:
print(len(df_scaled))
df_scaled = df_scaled.dropna()
print(len(df_scaled))

kmeans = KMeans(n_clusters=5, random_state=1).fit(df_scaled)
print(kmeans)

It's not immediately obvious what we can do with this `KMeans` object. But let's explore it. Use the tab autocomplete to see the different methods.

In [ ]:
kmeans.

The cluster centers are defined by the (standardized) value for each variable. Each observation is assigned to the cluster with the closest center.

In [ ]:
kmeans.cluster_centers_

Notice that the `cluster_centers_` is an array that is `K x L`, where `K` is the number of clusters and `L` is the number of variables.

Here, we have 5 clusters, and we used 10 variables to define the clusters.

In [ ]:
print(kmeans.cluster_centers_.shape)
print(len(df_scaled.columns))

The `labels_` attribute gives the label of the cluster to which each observation (i.e., each precinct) is assigned.

In [ ]:
kmeans.labels_

So there are as many labels as rows in our dataframe.

In [ ]:
print(kmeans.labels_.shape)
print(len(df_scaled))

That means that we can simply add the cluster id back to our original dataframe!

In [ ]:
df_scaled['cluster_id'] = kmeans.labels_

How large is each cluster? Note that the algorithm doesn't aim to produce equal-size groupings.

In [ ]:
df_scaled.groupby('cluster_id').size()

<div class="alert alert-block alert-info">
<h3>Key Takeaways</h3>
<ul>
  <li>Cluster analysis is an exploratory data tool</li>
  <li>They are an example of <em>data reduction</em>—reducing your data to something that can be readily interpreted</li>
</div>